<a href="https://colab.research.google.com/github/Nils1511/Weather-Recognition/blob/main/Dphi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![dphi banner](https://dphi-courses.s3.ap-south-1.amazonaws.com/Datathons/dphi_banner.png)

# **Getting Started Code For [Data Sprint #62](https://dphi.tech/challenges/datathon/) on DPhi**

## Download the images
We are given google drive link in the '[Data](https://drive.google.com/file/d/1G_Ix97_JP5NhcmMY74ctA2piDxbiU1Lk/view?usp=sharing)' section of problem page which has all the required train images (to build the model) and test images to predict the label of these images and submit the predictions on the [DPhi platform](https://drive.google.com/file/d/1G_Ix97_JP5NhcmMY74ctA2piDxbiU1Lk/view?usp=sharing).

We can use **GoogleDriveDownloader** form **google_drive_downloader** library in Python to download the shared files from the shared Google drive link: https://drive.google.com/file/d/1G_Ix97_JP5NhcmMY74ctA2piDxbiU1Lk/view?usp=sharing

The file id in the above link is: **1G_Ix97_JP5NhcmMY74ctA2piDxbiU1Lk**

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1G_Ix97_JP5NhcmMY74ctA2piDxbiU1Lk',
                                    dest_path='content/weather.zip',
                                    unzip=True)

Unzipping...

/usr/local/lib/python3.7/dist-packages/google_drive_downloader/google_drive_downloader.py:78: UserWarning: Ignoring `unzip` since "1G_Ix97_JP5NhcmMY74ctA2piDxbiU1Lk" does not look like a valid zip file
  warnings.warn('Ignoring `unzip` since "{}" does not look like a valid zip file'.format(file_id))


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
ls

content/  gdrive/  sample_data/


In [ ]:
!unzip('/content/gdrive/MyDrive/weather.zip')

/bin/bash: -c: line 0: syntax error near unexpected token `'/content/gdrive/MyDrive/weather.zip''
/bin/bash: -c: line 0: `unzip('/content/gdrive/MyDrive/weather.zip')'


In [ ]:
from zipfile import ZipFile

with ZipFile('/content/gdrive/MyDrive/weather.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('content')
   print('File is unzipped in content folder')

File is unzipped in content folder


In [ ]:
from zipfile import ZipFile

with ZipFile('', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('content')
   print('File is unzipped in content folder')

FileNotFoundError: ignored

We have all the files from the shared Google drive link downloaded in the colab environment.

## Loading Libraries
All Python capabilities are not loaded to our working environment by default (even they are already installed in your system). So, we import each and every library that we want to use.

We chose alias names for our libraries for the sake of our convenience (numpy --> np and pandas --> pd, tensorlow --> tf).

Note: You can import all the libraries that you think will be required or can import it as you go along.

In [ ]:
import pandas as pd                                     # Data analysis and manipultion tool
import numpy as np                                      # Fundamental package for linear algebra and multidimensional arrays
import tensorflow as tf                                 # Deep Learning Tool
import os                                               # OS module in Python provides a way of using operating system dependent functionality
import cv2                                              # Library for image processing
from sklearn.model_selection import train_test_split    # For splitting the data into train and validation set

## Loading and preparing training data
The train and test images are given in two different folders - 'train' and 'test'. The labels of train images are given in a csv file 'Train.csv' with respective image id (i.e. image file name).

#### Getting the labels of the images

In [ ]:
labels = pd.read_csv("/content/content/Training_set.csv")   # loading the labels
labels.head()           # will display the first five rows in labels dataframe

,filename,label
0,Image_1.jpg,sunrise
1,Image_2.jpg,shine
2,Image_3.jpg,cloudy
3,Image_4.jpg,shine
4,Image_5.jpg,sunrise


In [ ]:
labels.tail()            # will display the last five rows in labels dataframe

,filename,label
1043,Image_1044.jpg,foggy
1044,Image_1045.jpg,sunrise
1045,Image_1046.jpg,cloudy
1046,Image_1047.jpg,rainy
1047,Image_1048.jpg,sunrise


#### Getting images file path

In [ ]:
file_paths = [[fname, '/content/content/train/' + fname] for fname in labels['filename']]

#### Confirming if no. of labels is equal to no. of images

In [ ]:
# Confirm if number of images is same as number of labels given
if len(labels) == len(file_paths):
    print('Number of labels i.e. ', len(labels), 'matches the number of filenames i.e. ', len(file_paths))
else:
    print('Number of labels does not match the number of filenames')

Number of labels i.e.  1048 matches the number of filenames i.e.  1048


#### Converting the file_paths to dataframe

In [ ]:
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
images.head()

,filename,filepaths
0,Image_1.jpg,/content/content/train/Image_1.jpg
1,Image_2.jpg,/content/content/train/Image_2.jpg
2,Image_3.jpg,/content/content/train/Image_3.jpg
3,Image_4.jpg,/content/content/train/Image_4.jpg
4,Image_5.jpg,/content/content/train/Image_5.jpg


#### Combining the labels with the images

In [ ]:
train_data = pd.merge(images, labels, how = 'inner', on = 'filename')
train_data.head()       

,filename,filepaths,label
0,Image_1.jpg,/content/content/train/Image_1.jpg,sunrise
1,Image_2.jpg,/content/content/train/Image_2.jpg,shine
2,Image_3.jpg,/content/content/train/Image_3.jpg,cloudy
3,Image_4.jpg,/content/content/train/Image_4.jpg,shine
4,Image_5.jpg,/content/content/train/Image_5.jpg,sunrise


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_data['label'] = le.fit_transform(train_data['label'])

The 'train_data' dataframe contains all the image id, their locations and their respective labels. Now the training data is ready.

## Data Pre-processing
It is necessary to bring all the images in the same shape and size, also convert them to their pixel values because all machine learning or deep learning models accepts only the numerical data. Also we need to convert all the labels from categorical to numerical values.

In [ ]:
data = []     # initialize an empty numpy array
image_size = 100      # image size taken is 100 here. one can take other size too
for i in range(len(train_data)):
  
  img_array = cv2.imread(train_data['filepaths'][i], cv2.IMREAD_GRAYSCALE)   # converting the image to gray scale

  new_img_array = cv2.resize(img_array, (image_size, image_size))      # resizing the image array
  data.append([new_img_array, train_data['label'][i]])

In [ ]:
# image pixels of a image
data[5]

[array([[129, 130, 131, ..., 129, 128, 127],
        [129, 130, 131, ..., 129, 128, 128],
        [129, 130, 131, ..., 130, 130, 129],
        ...,
        [ 14,  14,  14, ...,  18,  18,  18],
        [ 16,  13,  14, ...,  17,  17,  17],
        [ 24,  11,  13, ...,  16,  17,  17]], dtype=uint8), 0]

#### Shuffle the data

In [ ]:
np.random.shuffle(data)

#### Separating the images and labels


In [ ]:
x = []
y = []
for image in data:
  x.append(image[0])
  y.append(image[1])

# converting x & y to numpy array as they are list
x = np.array(x)
y = np.array(y)

In [ ]:
np.unique(y, return_counts=True)

(array([0, 1, 2, 3, 4]), array([210, 210, 209, 174, 245]))

#### Splitting the data into Train and Validation Set
We want to check the performance of the model that we built. For this purpose, we always split (both independent and dependent data) the given data into training set which will be used to train the model, and test set which will be used to check how accurately the model is predicting outcomes.

For this purpose we have a class called 'train_test_split' in the 'sklearn.model_selection' module.

In [ ]:
x =  x.reshape(-1, 100, 100, 1)

In [ ]:
# split the data
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.3, random_state = 42)

## Building Model
Now we are finally ready, and we can train the model.

There are many machine learning or deep learning models like Random Forest, Decision Tree, Multi-Layer Perceptron (MLP), Convolution Neural Network (CNN), etc. to say you some.


Then we would feed the model both with the data (X_train) and the answers for that data (y_train)

In [ ]:
cnn = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(100, 100, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # tf.keras.layers.Flatten(input_shape=(100, 100, 1)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='sigmoid')
])

cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 33856)             0         
                                                                 
 dense (Dense)               (None, 64)                2

In [ ]:
cnn.compile(optimizer='AdaDelta',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
cnn.fit(X_train, y_train, epochs=20, batch_size=9)                       #for highest epochs=20, batchsize=9 submission(2)

Epoch 1/20
82/82 [==============================] - 10s 116ms/step - loss: 7.5897 - accuracy: 0.2347
Epoch 2/20
82/82 [==============================] - 9s 114ms/step - loss: 5.8218 - accuracy: 0.2742
Epoch 3/20
82/82 [==============================] - 9s 113ms/step - loss: 5.1968 - accuracy: 0.3042
Epoch 4/20
82/82 [==============================] - 9s 113ms/step - loss: 4.7663 - accuracy: 0.3206
Epoch 5/20
82/82 [==============================] - 9s 113ms/step - loss: 4.4352 - accuracy: 0.3424
Epoch 6/20
82/82 [==============================] - 9s 113ms/step - loss: 4.1544 - accuracy: 0.3643
Epoch 7/20
82/82 [==============================] - 9s 113ms/step - loss: 3.9109 - accuracy: 0.3670
Epoch 8/20
82/82 [==============================] - 9s 114ms/step - loss: 3.7281 - accuracy: 0.4011
Epoch 9/20
82/82 [==============================] - 9s 115ms/step - loss: 3.5385 - accuracy: 0.3984
Epoch 10/20
82/82 [==============================] - 10s 119ms/step - loss: 3.3609 - accuracy: 0.40

## Validate the model
Wonder🤔 how well your model learned! Lets check its performance on the X_val data.

In [ ]:
cnn.evaluate(X_val, y_val)

10/10 [==============================] - 1s 86ms/step - loss: 2.8137 - accuracy: 0.4413


[2.813659191131592, 0.4412698447704315]

## Predict The Output For Testing Dataset 😅
We have trained our model, evaluated it and now finally we will predict the output/target for the testing data (i.e. Test.csv).

#### Load Test Set
Load the test data on which final submission is to be made.

In [ ]:
# Loading the order of the image's name that has been provided
test_image_order = pd.read_csv("/content/content/Testing_set.csv")
test_image_order.head()

,filename
0,Image_1.jpg
1,Image_2.jpg
2,Image_3.jpg
3,Image_4.jpg
4,Image_5.jpg


#### Getting images file path

In [ ]:
file_paths = [[fname, '/content/content/test/' + fname] for fname in test_image_order['filename']]

#### Confirm if number of images in test folder is same as number of image names in 'Testing_set_face_mask.csv'

In [ ]:
# Confirm if number of images is same as number of labels given
if len(test_image_order) == len(file_paths):
    print('Number of image names i.e. ', len(test_image_order), 'matches the number of file paths i.e. ', len(file_paths))
else:
    print('Number of image names does not match the number of filepaths')

Number of image names i.e.  450 matches the number of file paths i.e.  450


#### Converting the file_paths to dataframe

In [ ]:
test_images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
test_images.head()

,filename,filepaths
0,Image_1.jpg,/content/content/test/Image_1.jpg
1,Image_2.jpg,/content/content/test/Image_2.jpg
2,Image_3.jpg,/content/content/test/Image_3.jpg
3,Image_4.jpg,/content/content/test/Image_4.jpg
4,Image_5.jpg,/content/content/test/Image_5.jpg


## Data Pre-processing on test_data


In [ ]:
test_pixel_data = []     # initialize an empty numpy array
image_size = 100      # image size taken is 100 here. one can take other size too
for i in range(len(test_images)):
  
  img_array = cv2.imread(test_images['filepaths'][i], cv2.IMREAD_GRAYSCALE)   # converting the image to gray scale

  new_img_array = cv2.resize(img_array, (image_size, image_size))      # resizing the image array

  test_pixel_data.append(new_img_array)

In [ ]:
test_pixel_data = np.array(test_pixel_data)

In [ ]:
test_pixel_data =  test_pixel_data.reshape(-1, 100, 100, 1)

### Make Prediction on Test Dataset
Time to make a submission!!!

In [ ]:
pred = cnn.predict(test_pixel_data)

In [ ]:
# The predicted values are the probabilities value
pred[0]

array([7.0917606e-04, 1.7997722e-07, 9.9981070e-01, 9.9528408e-01,
       1.4144942e-06], dtype=float32)

In [ ]:
np.argmax(pred[0])

2

The above values are probability values. We need to convert it into respective classes. We can use np.argmax for the same.

In [ ]:
prediction = []
for value in pred:
  prediction.append(np.argmax(value))

In [ ]:
predictions = le.inverse_transform(prediction)

## **How to save prediciton results locally via jupyter notebook?**
If you are working on Jupyter notebook, execute below block of codes. A file named 'submission.csv' will be created in your current working directory.

In [ ]:
res = pd.DataFrame({'filename': test_images['filename'], 'label': predictions})  # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False)      # the csv file will be saved locally on the same location where this notebook is located.

# **OR,**
**If you are working on Google Colab then use the below set of code to save prediction results locally**

## **How to save prediction results locally via colab notebook?**
If you are working on Google Colab Notebook, execute below block of codes. A file named 'prediction_results' will be downloaded in your system.

In [ ]:
res = pd.DataFrame({'filename': test_images['filename'], 'label': predictions})  # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False) 

# To download the csv file locally
from google.colab import files        
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>